# Python: ineragir avec les base de données avec SQLAlchemy

**[SQLAlchemy](https://docs.sqlalchemy.org/en/20/tutorial/index.html#unified-tutorial)** est un ensemble complet d'outils pour travailler avec des bases de données et Python.  

Composants Principaux :
- **Engine** : Point de connexion à la base de données
- **Session** : Zone de travail pour les transactions
- **Declarative Base** : Classe de base pour les modèles
- **Modèles** : Classes Python représentant des tables
- **Relations** : Liens entre modèles (One-to-Many, Many-to-Many)

- Installer la bibliothèque SQLAlchemy

In [1]:
# code ici
! pip install sqlalchemy

- Verifier la version de sqlalchemy installée

In [2]:
# code ici
import sqlalchemy
sqlalchemy.__version__

'2.0.40'

#### Connexion à une base de données

- La fonction `create_engine(url)` permet de se connecter à une base de données via une URL de connexion.
 > La classe **Engine** représente une interface de base pour la base de données, servant de point de départ pour toute interaction.

 - Importer la fonction `create_engine`de `sqlalchemy`.

In [3]:
from sqlalchemy import create_engine

- - Créer une connexion à la base de données `'sqlite:///biblio.db'` puis l'affecter à `engine`. Afficher le type de `engine`.


In [4]:
engine = create_engine('sqlite:///biblio.db')
type(engine)

sqlalchemy.engine.base.Engine

Ici, nous définissons des constructions au niveau du module qui formeront les structures que nous interrogerons dans la base de données. Cette structure, appelée mappage déclaratif, définit à la fois un modèle objet Python et des métadonnées décrivant les tables SQL réelles qui existent ou existeront dans une base de données particulière :

#### Définition d'une entité

- La classe `DeclarativeBase` est une structure appelée **mappage déclaratif**,  qui définit à la fois un modèle d'objet Python, ainsi que des métadonnées de base de données qui correspondent à de véritables tables SQL qui existent, ou existeront, dans une base de données.

- La classe `Mapped` fournit les informations appropriées aux vérificateurs de type afin que les attributs mappés ORM soient correctement typés.
 - Exemple: `Mapped[int]` permet de spécifier que l'attribut mappé est du type entier.

- La fonction [`mapped_column()`](https://docs.sqlalchemy.org/en/20/orm/mapping_api.html#sqlalchemy.orm.mapped_column) est utilisée pour faire corespondre un attribut à une colonne de table.
 - Exemple: `_id: Mapped[int] = mapped_column(primary_key=True)` permet de faire corespondre l'attribut `_id` à la clé primaire de la tables.

[source](https://docs.sqlalchemy.org/en/20/orm/dataclasses.html#orm-declarative-native-dataclasses)

In [5]:
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import MappedAsDataclass


class Base(MappedAsDataclass, DeclarativeBase):
    """subclasses will be converted to dataclasses"""

class User(Base):
    __tablename__ = "user"
    id: Mapped[int] = mapped_column(init=True, primary_key=True)

#### Création de toutes les entités définies dans vos modèles

`Base.metadata.create_all(engine)` permet de créer dans notre base de données (`engine`) toutes les tables représenter par des classe qui ont hérité de la classe `Base`.

En utilisant nos métadonnées de table et notre moteur, nous pouvons générer notre schéma immédiatement dans notre base de données SQLite cible, en utilisant une méthode appelée MetaData.create_all() :

In [ ]:
# code ici

In [ ]:
Base.metadata.create_all(engine)

#### Créer une classe `Address` qui représente la table Address dans la base de données.
Attributs:
 - _id (int): id d'une adresse dans la base de données
 - email (str): une adresse mail

In [5]:
# code ici

In [8]:
class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)

In [ ]:
Base.metadata.create_all(engine)

#### Configuration des attributs: Explorons la fonction `mapped_column`

 mapped_column() prend en charge des options de champ pour une attribut, notamment. La fonctionnalité est équivalente à celle des dataclasses :

- `init` (bool) : `True` par défaut, indique si l'attribut doit faire partie du constructeur `__init__()` (`True`) ou pas (`False`).

- `default`: indique que le paramètre pour ce attribut est optionnel dans le constructeur `__init__`.

- `insert_default`: indique une valeur par défaut pour un attribut dans le constructeur `__init__()`.

- `repr` (bool): `True` par défaut, indique que le champ doit faire partie de la méthode `__repr__()` générée.

In [19]:
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy import String
from datetime import datetime
from sqlalchemy import func
from sqlalchemy.orm import MappedAsDataclass


class Base(MappedAsDataclass, DeclarativeBase):
    """subclasses will be converted to dataclasses"""


class User(Base):
    __tablename__ = "user"
    _id: Mapped[int] = mapped_column(init=False, primary_key=True, repr=False)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[str|None] = mapped_column(default=None)
    created_at: Mapped[datetime] = mapped_column(
        insert_default=datetime.now, default=None
    )

#print(User(name="Monsia", fullname="DOugban Monsia"))

In [20]:
Base.metadata.create_all(engine)

#### Enrichire la classe `Address`
Attributs:
 - tel (str): le numero de téléphone
 - city (str): la ville.
 - district (str): le quartier
 - street (str): la rue (optionnel)
 - house (int): numero de de la maison

In [8]:
# code ici
class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    tel: Mapped[str]
    city: Mapped[str]
    district: Mapped[str]
    street: Mapped[str]
    house: Mapped[int]

In [9]:
Base.metadata.create_all(engine)

#### Configuration des relations entre entités

- La classe `ForeignKey("class_name.id_name")`: définit une dépendance entre deux colonnes. classe_name désigne le nom de la classe, id_name définit l'identifiant.

- La fonction `relationship(...)`: définit une relation entre deux classes mappées. Il existe trois types de rélation:
 - **[Many to One](https://docs.sqlalchemy.org/en/20/orm/basic_relationships.html#many-to-one)**: utilise une clé étrangère dans la table de cardinalité 1 faisant référence à table de cardinalité n.

 - **[One To One](https://docs.sqlalchemy.org/en/20/orm/basic_relationships.html#one-to-one)** est essentiellement une relation **One To Many** du point de vue de la clé étrangère.

 - **[Many to Many](https://docs.sqlalchemy.org/en/20/orm/basic_relationships.html#many-to-many)**:  utilise une table d'association entre deux classes. La table d'association est objet de type `Table` et est indiquée par l'argument `secondary` dans `relationship()`.

In [ ]:
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy import String
from datetime import datetime
from sqlalchemy import func
from sqlalchemy.orm import MappedAsDataclass
from sqlalchemy.orm import relationship
from sqlalchemy import ForeignKey
from typing import List


class Base(MappedAsDataclass, DeclarativeBase):
    """subclasses will be converted to dataclasses"""
    pass


class User(Base):
    __tablename__ = "user"
    _id: Mapped[int] = mapped_column(init=False, primary_key=True, repr=False)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[str|None] = mapped_column(default=None)
    created_at: Mapped[datetime] = mapped_column(
        insert_default=datetime.now, default=None
    )
    # address_id: Mapped[int] = mapped_column(ForeignKey("address._id"), default=None)
    addresses: Mapped[List["Address"] | None] = relationship(
        back_populates="user", cascade="all, delete-orphan", default=None
    )

class Address(Base):
    __tablename__ = "address"
    _id: Mapped[int] = mapped_column(init=False, primary_key=True)
    email: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user._id"), default=None)
    user: Mapped["User"] = relationship(b ack_populates="addresses", default=None)




In [15]:
Base.metadata.create_all(engine)

#### insérer des données

In [16]:
from sqlalchemy.orm import Session

In [17]:
with Session(engine) as session:
    user_1 = User(
        name="Koffi",
        fullname="Aya Sandrine",
        addresses=[
            Address(email="sandrine.koffi@data354.co"),
            Address(email="sandrine@orange.ci"),
        ],
    )

    user_2 = User(name="Koné", fullname="Patrick", addresses=[])

    session.add_all([user_1, user_2])
    session.commit()

OperationalError: (sqlite3.OperationalError) no such column: _id
[SQL: INSERT INTO user (name, fullname, created_at) VALUES (?, ?, ?) RETURNING _id]
[parameters: ('Koffi', 'Aya Sandrine', '2025-05-14 02:05:40.595969')]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

- Insérer dans la table `Address l'adresse de Koné Patrique: email = patrick.kone@gmail.com.


In [ ]:
# code ici

#### Lecture

In [ ]:
koffi = session.query(User).filter_by(name="Koffi").first()
print(koffi)

- Lire les informations de Koné les affecter dans la variable kone.

In [ ]:
# code ici

#### Mise à Jour

In [ ]:
koffi.fullname = f"{koffi.fullname} Stéphanie"
session.commit()

In [ ]:
koffi = session.query(User).filter_by(name="Koffi").first()
print(koffi)

- Mettre à jour l'adresse de Patrick

In [ ]:
# code ici

#### Suppression

In [ ]:
session.delete(koffi)
session.commit()

- Supprimer l'utilisateur Patrick

In [ ]:
# code ici

### Exercice

1. Créer un module `database.py` qui contiendra les configuration de connexion à la base de données.
2. Modeliser la base de données ci-dessous dans un module `models.py`
3. Effectuer les operation de CRUD dans un interpréteur python

#### Table `Client`

| ID_Client | Nom        | Prénom  | Email                     | Téléphone     | Adresse                          |
|-----------|------------|---------|---------------------------|---------------|----------------------------------|
| 1         | Dupont     | Jean    | jean.dupont@email.com     | 0612345678    | 12 Rue de la Paix, Paris         |
| 2         | Martin     | Sophie  | sophie.martin@email.com   | 0698765432    | 25 Avenue des Champs, Lyon       |
| 3         | Leroy      | Pierre  | pierre.leroy@email.com    | 0687654321    | 8 Boulevard Voltaire, Marseille  |
| 4         | Dubois     | Marie   | marie.dubois@email.com    | 0678912345    | 15 Rue du Commerce, Lille        |
| 5         | Bernard    | Thomas  | thomas.bernard@email.com  | 0632145698    | 42 Avenue Foch, Bordeaux         |
| 6         | Petit      | Alice   | alice.petit@email.com     | 0698745632    | 7 Rue de la République, Nantes   |
| 7         | Moreau     | Julien  | julien.moreau@email.com   | 0654789123    | 33 Rue Victor Hugo, Toulouse     |
| 8         | Laurent    | Camille | camille.laurent@email.com | 0645897123    | 19 Boulevard Gambetta, Nice      |
| 9         | Simon      | Luc     | luc.simon@email.com       | 0678451296    | 5 Rue Pasteur, Strasbourg        |
| 10        | Michel     | Élodie  | elodie.michel@email.com   | 0696325418    | 22 Avenue Jean Jaurès, Montpellier |

#### Table `Produit`

| ID_Produit | Nom_Produit          | Catégorie       | Prix    | Stock |
|------------|----------------------|-----------------|---------|-------|
| 101        | Ordinateur Portable  | Informatique    | 899.99  | 50    |
| 102        | Smartphone Pro       | Électronique    | 699.99  | 100   |
| 103        | Roman Best-Seller    | Livres          | 19.99   | 200   |
| 104        | Casque Bluetooth     | Audio           | 129.99  | 75    |
| 105        | Montre Connectée     | Accessoires     | 249.99  | 60    |
| 106        | Cafetière Électrique | Électroménager  | 59.99   | 120   |
| 107        | Sac à Dos            | Mode            | 39.99   | 150   |
| 108        | Enceinte Portable    | Audio           | 89.99   | 80    |
| 109        | Baskets de Sport     | Chaussures      | 79.99   | 90    |
| 110        | Écran 24"            | Informatique    | 179.99  | 40    |

#### Table `Commande`

| ID_Commande | ID_Client | Date_Commande  | Montant_Total | Statut       |
|-------------|-----------|----------------|---------------|--------------|
| 1001        | 3         | 2023-05-15     | 919.98        | Livré        |
| 1002        | 7         | 2023-05-16     | 249.99        | Expédié      |
| 1003        | 1         | 2023-05-17     | 129.99        | En préparation |
| 1004        | 5         | 2023-05-18     | 59.99         | Livré        |
| 1005        | 2         | 2023-05-19     | 699.99        | Expédié      |
| 1006        | 10         | 2023-05-20     | 179.99        | En attente   |
| 1007        | 3         | 2023-05-21     | 39.99         | Livré        |
| 1008        | 3         | 2023-05-22     | 89.99         | Expédié      |
| 1009        | 6         | 2023-05-23     | 79.99         | En préparation |
| 1010        | 10        | 2023-05-24     | 199.98        | En attente   |

## Table `Ligne_Commande` (optionnelle)

| ID_Ligne | ID_Commande | ID_Produit | Quantité | Prix_Unitaire |
|----------|-------------|------------|----------|---------------|
| 1        | 1001        | 101        | 1        | 899.99        |
| 2        | 1001        | 107        | 1        | 39.99         |
| 3        | 1002        | 105        | 1        | 249.99        |
| 4        | 1003        | 104        | 1        | 129.99        |
| 5        | 1004        | 106        | 1        | 59.99         |
| 6        | 1005        | 102        | 1        | 699.99        |
| 7        | 1006        | 110        | 1        | 179.99        |
| 8        | 1007        | 107        | 1        | 39.99         |
| 9        | 1008        | 108        | 1        | 89.99         |
| 10       | 1009        | 109        | 1        | 79.99         |